This notebook assumes access to denoised, detrended signals and calculates the appropriate parameters for the signal.

In [1]:
%run "Data_PreProcessing.ipynb" #allowing access to the filtered database with preprocessed signals

100%|████████████████████████████████████████████████████████████████████████████████| 229/229 [00:02<00:00, 91.69it/s]


In [2]:
from scipy.signal import find_peaks #for peak finding

In [3]:
def peak_to_peak(signal, distance):
    peaks, _ = find_peaks(signal, distance = distance)
    peak_to_peaks = peaks[1:-1]

    # Calculate differences between adjacent peak indices
    peak_distances = np.diff(peak_to_peaks)

    # Compute the average distance between peaks
    peak_av = np.mean(peak_distances)
    #peak_std = np.std(peak_distances) dont think we want this??
    
    return peak_av

peak_to_peak_averages = np.zeros(shape = no_patients)
for i, signal in enumerate(butterworth_detrended_signals):
    peak_av = peak_to_peak(signal, 600)
    peak_to_peak_averages[i] = peak_av